# Simulating phenotypes

- Mock example with very small sample size and number of SNPs
- Simple example using 2 components (genetic and noise), and bi-allelic SNPs
- Frequencies to sample alleles: 0.1, 0.2, 0.3

# Functions

In [212]:
library(data.table)

simGenotype = function(N = 50, nSNP = 30, frequencies = c(0.1, 0.2, 0.4)) {
    sps = paste0("%0", nchar(as.character(N)), "d")
    spsn = paste0("%0", nchar(as.character(nSNP)), "d")
    
    samples = paste0("ID_", sprintf(sps, 1:N))
    snps = paste0("SNP_", sprintf(spsn, 1:nSNP))
    freq = sample(frequencies, nSNP, replace = TRUE)
    X <- sapply(1:nSNP, function(x) rbinom(N, 2, freq[x]))
    colnames(X) <- snps
    rownames(X) <- samples
    return(X)
}


geneticEffects = function (genotype, causalSNP = 10, mBeta = 0, sdBeta = 1) {
    cSNP = sort(sample(colnames(genotype), causalSNP, replace = FALSE))
    betas = rnorm(causalSNP, mBeta, sdBeta)
    g = colnames(genotype)
    effects = rep(0, length(g))
    effects[g %in% cSNP] = betas
    names(effects) = g
    return(effects)
}

createPairs = function(trait, random_mating = 0.5, groups = 10) {
    tt = trait[, .(id, trait)]
    tt[, g := cut(trait , quantile(trait, probs = (0:groups)/groups),
        labels = FALSE, include.lowest = TRUE)]

    pairs = NULL
    for (i in 1:nrow(tt)) {
        if (nrow(tt) >= 2) {
            if (runif(1) < random_mating) {
                cc = tt[sample(.N, 2)]$id
                tt = tt[!id %in% cc]
                pairs = rbind(pairs, as.vector(cc))}
            else {
                sid = tt[sample(.N, 1)]$id
                sg = tt[id == sid]$g
                tt = tt[!id %in% sid]
                aid = tt[g == sg][sample(.N, 1)]$id
                if (length(aid) == 0) { aid = tt[sample(.N, 1)]$id }
                tt = tt[!id %in% aid]
                pairs = rbind(pairs, c(sid, aid))
            }
        }
    }
    return(pairs)
}


createCouples = function(genotype, trait, random_mating = 0.5) {
    
    temp = rownames(genotype)
    pairs = createPairs(trait, random_mating = random_mating)
    couples = list()
    snps = colnames(genotype)
    for (i in 1:nrow(pairs)) {
        temp = data.table(t(genotype[pairs[i,], ]))
        rownames(temp) = snps 
        couples[[i]] = temp
    }
    return(couples)
}


reproduce = function(couples, nkids = 2) {

    # mendelian rules (very important)
    m = matrix(c(1, 0, 0, 0.5, 0.5, 0, 0, 1, 0, 0.25, 0.50, 0.25, 0, 0.50, 0.5, 0, 0, 1), 
        nrow = 6, ncol = 3, byrow  = TRUE)       
    colnames(m) = c(0, 1, 2)
    rownames(m) = c("00", "01", "02", "11", "12", "22")
    
    kids = list()
    
    family = data.table()
    j = 1

    for (i in seq_along(couples)) {
        for (ii in 1:nkids) {
            temp = couples[[i]]
            fids = colnames(temp)
            temp$k = temp[, apply(.SD, 1, getKidGenotype, matrix = m)]
            temp = temp[, .(k)]
            setnames(temp, "k", paste0("ID_", j))
            temp[, snp := rownames(couples[[1]])]
            kids[[paste0(i, ii)]] = temp

            family = rbind(family, data.table(father = fids[1], mother = fids[2], kid = paste0("ID_", j)))
            j = j+1
        }
    }

    kk = Reduce(function(...) merge(...,  by = "snp"), kids)
    kk[, snp := NULL]
    kk = as.matrix(t(kk))
    colnames(kk) = rownames(couples[[1]])
    return(list(kk, family))
}


# function to get offspring genotype
getKidGenotype = function(values, matrix) {
    comba = paste0(values, collapse = "")
    combb = paste0(rev(values), collapse = "")
    s = grep(paste0(comba, "|", combb), rownames(matrix))
    prob = as.vector(matrix[s, ])
    return(sample(0:2, size = 1, prob = prob))
}


simKidTrait = function(kids, ge) {
    return(kids[[1]] %*% ge)
}


getNoiseComponent = function(genotype, mNoise = 0, sdNoise = 1) {
    n = nrow(genotype)
    return(rnorm(n, mNoise, sdNoise))
}


rescaleVar = function(component, prop) {
    component = as.vector(component) 
    var_component = var(component)
    scale_factor = sqrt(prop/var_component)
    return(component * scale_factor)
}


scalingFactor = function(component, prop) {
    component = as.vector(component) 
    var_component = var(component)
    return(sqrt(prop/var_component))
}

# Initial example
- 1000 people
- 100 snps
- 50 causal

In [84]:
gen = 0.3
noise = 1 - gen

genotype = simGenotype(1000, 300)
ge = geneticEffects(genotype, 100)
parentGComp = genotype %*% ge
parentNComp = getNoiseComponent(genotype)

pgsf = scalingFactor(parentGComp, gen)
pnsf = scalingFactor(parentNComp, noise)
parentTrait = parentGComp * pgsf + parentNComp * pnsf
dparentTrait = data.table(id = rownames(parentTrait), trait = parentTrait[, 1])

In [85]:
head(genotype)

,SNP_001,SNP_002,SNP_003,SNP_004,SNP_005,SNP_006,SNP_007,SNP_008,SNP_009,SNP_010,⋯,SNP_291,SNP_292,SNP_293,SNP_294,SNP_295,SNP_296,SNP_297,SNP_298,SNP_299,SNP_300
ID_0001,0,0,0,1,1,0,0,0,0,1,⋯,0,1,0,0,0,0,0,0,0,1
ID_0002,1,0,0,1,0,1,0,0,0,1,⋯,0,0,1,0,0,0,0,0,0,2
ID_0003,1,0,0,1,0,1,0,0,0,1,⋯,1,1,0,0,1,0,0,0,2,0
ID_0004,1,1,0,1,1,0,0,0,1,1,⋯,0,0,0,1,0,0,0,0,1,1
ID_0005,2,1,0,2,1,2,1,0,1,0,⋯,1,1,0,0,0,1,0,0,1,0
ID_0006,0,1,1,1,0,0,0,0,0,1,⋯,0,2,0,0,0,0,1,0,1,1


In [88]:
head(kids[[1]])

,SNP_001,SNP_002,SNP_003,SNP_004,SNP_005,SNP_006,SNP_007,SNP_008,SNP_009,SNP_010,⋯,SNP_291,SNP_292,SNP_293,SNP_294,SNP_295,SNP_296,SNP_297,SNP_298,SNP_299,SNP_300
ID_1,0,0,0,0,2,1,1,0,0,0,⋯,1,0,1,0,0,1,0,0,1,1
ID_2,0,0,0,1,2,1,1,1,0,0,⋯,1,1,1,0,1,0,1,0,1,0
ID_3,1,0,0,0,0,0,0,0,0,1,⋯,1,1,1,0,0,0,0,0,2,0
ID_4,1,0,0,0,1,0,1,0,0,0,⋯,0,1,1,0,0,0,0,0,1,1
ID_5,1,0,0,2,1,2,0,1,0,0,⋯,1,1,0,0,0,1,1,0,1,0
ID_6,1,1,0,0,1,2,0,2,1,0,⋯,1,2,0,2,0,1,0,0,1,1


In [90]:
# correlation between phenotypes
cor(dkidTrait[, .(trait, tfather, tmother)])

,trait,tfather,tmother
trait,1.0000000,0.18313253,0.15145904
tfather,0.1831325,1.00000000,0.01713349
tmother,0.1514590,0.01713349,1.00000000


In [91]:
# variance composition parents
paste0("Genetic part: ", round(var(parentGComp[, 1] * pgsf)/var(parentTrait[, 1]), 2))
paste0("Noise part: ", round(var(parentNComp * pnsf) /var(parentTrait[, 1]), 2))

[1] "Genetic part: 0.29"

[1] "Noise part: 0.68"

In [92]:
# variance composition kids
paste0("Genetic part: ", round(var(kidGComp[, 1] * pgsf)/var(kidTrait[, 1]), 2))
paste0("Noise part: ", round(var(kidNComp * pnsf) /var(kidTrait[, 1]), 2))

[1] "Genetic part: 0.27"

[1] "Noise part: 0.7"

# Several generations

In [255]:
# function to create several generations
# very inefficient function
createGenerations = function(nGen = 3, N = 1000, nSNP = 100, causalSNP = 50, genVar = 0.3, random_mating = 1) {

    noise = 1 - genVar
    genotype = simGenotype(N, nSNP)
    ge = geneticEffects(genotype, causalSNP)

    traits = list()

    parentGComp = genotype %*% ge
    parentNComp = getNoiseComponent(genotype)

    pgsf = scalingFactor(parentGComp, gen)
    pnsf = scalingFactor(parentNComp, noise)
    parentTrait = parentGComp * pgsf + parentNComp * pnsf
    dparentTrait = data.table(id = rownames(parentTrait), trait = parentTrait[, 1])
    
    kids = NULL
    for (i in 1:nGen) {
        print(paste0("::::::: creating generation: ", i))
        if (i == 1) {
            couples = createCouples(genotype, dparentTrait, random_mating = random_mating)
            kids = reproduce(couples)
            # genetic part
            kidGComp = kids[[1]] %*%  ge
            pgsf = scalingFactor(parentGComp, gen)
            pnsf = scalingFactor(parentNComp, noise)
            # noise part
            kidNComp = getNoiseComponent(kids[[1]])
            # trait
            kidTrait = kidGComp * pgsf + kidNComp * pnsf
            dkidTrait = data.table(id = rownames(kidTrait), trait = kidTrait[, 1])
            # merge traits
            family = kids[[2]]
            dkidTrait = merge(dkidTrait, family, by.x = "id", by.y = "kid")
            dkidTrait = merge(dkidTrait, dparentTrait[, .(id, father_trait = trait)], by.x = "father", by.y = "id", all.x = TRUE)
            dkidTrait = merge(dkidTrait, dparentTrait[, .(id, mother_trait = trait)], by.x = "mother", by.y = "id", all.x = TRUE)
            traits[[i]] = dkidTrait
        } else {
            couples = createCouples(kids[[1]], dkidTrait, random_mating = random_mating)
            kids = reproduce(couples)
            dparentTrait = dkidTrait
            # genetic part
            kidGComp = kids[[1]] %*%  ge
            # noise part
            kidNComp = getNoiseComponent(kids[[1]])
            # trait
            kidTrait = kidGComp * pgsf + kidNComp * pnsf
            dkidTrait = data.table(id = rownames(kidTrait), trait = kidTrait[, 1])
            # merge traits
            family = kids[[2]]
            dkidTrait = merge(dkidTrait, family, by.x = "id", by.y = "kid")
            dkidTrait = merge(dkidTrait, dparentTrait[, .(id, father_trait = trait)], by.x = "father", by.y = "id", all.x = TRUE)
            dkidTrait = merge(dkidTrait, dparentTrait[, .(id, mother_trait = trait)], by.x = "mother", by.y = "id", all.x = TRUE)
            traits[[i]] = dkidTrait
        }   
    }
    return(rbindlist(traits, idcol = "generation"))
}

# Some assortative mating

In [250]:
tt = createGenerations(nGen = 10,  N = 3000, nSNP = 500, causalSNP = 100, random_mating = 0.80)

In [256]:
dtt = tt[, .(average = mean(trait), 
    cor_kid_father = cor(trait, father_trait), 
    cor_kid_mother = cor(trait, mother_trait), 
    cor_father_mother = cor(father_trait, mother_trait)), generation]
dtt

generation,average,cor_kid_father,cor_kid_mother,cor_father_mother
<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.1883019,0.1999825,0.1886230,0.1741984
2,0.1620189,0.1604917,0.1731871,0.2483799
3,0.1780061,0.1639263,0.1659964,0.2881601
4,0.1854883,0.1978828,0.2234411,0.2203220
5,0.1567797,0.2037965,0.2199690,0.2381033
6,0.1675324,0.2143007,0.2171608,0.2124595
7,0.2012330,0.1229418,0.1578948,0.2134461
8,0.2095405,0.1840035,0.1755964,0.2605650
9,0.2172323,0.2042063,0.2170359,0.1891421


# Random mating

There seems to be issues with the scaling.

In [ ]:
cc = createGenerations(nGen = 10,  N = 3000, nSNP = 500, causalSNP = 100, random_mating = 1.0)

In [259]:
dcc = cc[, .(average = mean(trait), 
    cor_kid_father = cor(trait, father_trait), 
    cor_kid_mother = cor(trait, mother_trait), 
    cor_father_mother = cor(father_trait, mother_trait)), generation]
dcc

generation,average,cor_kid_father,cor_kid_mother,cor_father_mother
<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.3984700,0.1167146,0.1683724,-0.00397079
2,0.3973159,0.1271531,0.1502796,-0.01773577
3,0.4064284,0.1591879,0.1452047,0.07010770
4,0.4305984,0.1582951,0.1451794,-0.01904595
5,0.4139249,0.1339551,0.1672606,-0.02494781
6,0.4416047,0.1442953,0.1671510,-0.02755566
7,0.4043321,0.1454876,0.1451702,0.04670183
8,0.4210788,0.1261394,0.1620259,0.03053941
9,0.4333460,0.1436480,0.1299861,-0.02521360
